In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from fastai.vision import *
from fastai.widgets import *
from fastai.callbacks import *
from fastai.text import *
import os
import pickle

In [0]:
#load test.csv for nlp
test = pd.read_csv('/content/drive/My Drive/test_tabular_c4.csv', index_col=0)

#load df2.csv for tabular
test_table = pd.read_csv('/content/drive/My Drive/tabular_model/test_rolf.csv', index_col=0)

test_table does not contain one image -> deleted that image which is e74dd0579-1.jpg

test_table does not contain 27 rows of test_c4_tabular.csv

In [0]:
#add suffix to petid to match description field with image
test['image'] = test['PetID'] + '-1.jpg'
test_table['image'] = test_table['PetID'] + '-1.jpg'

In [0]:
#some rows are not present in test_table so drop
test_tabular = test_table.merge(test[['image','Description']], on='image', how='inner')

In [0]:
#drop adoptionspeed, not needed for prediction
test.drop('AdoptionSpeed', inplace=True, axis=1)
test_tabular.drop(['AdoptionSpeed', 'RescuerID'], inplace=True, axis=1)

In [28]:
model_path = '/content/drive/My Drive/models/learner_model/'
nlp_model_path = '/content/drive/My Drive/language_models/'
table_model_path = '/content/drive/My Drive/tabular_model/'

resnet34_learner = load_learner(model_path, "rs34_model.pkl")
densenet121_learner = load_learner(model_path, "dn121_model.pkl")
vgg_learner = load_learner(model_path, "vgg19_model.pkl")

nlp_learner = load_learner(nlp_model_path, "nlp_export.pkl")

table_learner = pickle.load(open(table_model_path + 'new_model.sav', 'rb'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
test_path = '/content/drive/My Drive/profile_picture_test/'

def ensemble_prediction(test_img, match_column):
    #select image, text and table for corresponding image id
    img = open_image(test_path + test_img)
    text = test.loc[test[match_column] == test_img, 'Description'].values
    row = test_tabular.loc[test_tabular[match_column] == test_img, ~test_tabular.columns.isin(['PetID', 'image', 'Description'])].values

    #make predictions
    resnet34_prediction = resnet34_learner.predict(img)
    densenet121_prediction = densenet121_learner.predict(img)
    vgg_prediction = vgg_learner.predict(img)
    
    nlp_prediction = nlp_learner.predict(text)

    table_prediction = table_learner.predict_proba(row)

    #ensemble average
    sum_pred = resnet34_prediction[2] + densenet121_prediction[2] + vgg_prediction[2] + nlp_prediction[2] + table_prediction
    prediction = sum_pred / 5
    
    #prediction results
    predicted_label = np.argmax(prediction).item()+1
    
    return predicted_label

#to give np array the correct style
submission_data = np.array([['dummy', 0]])

In [52]:
#progress bar
with tqdm(total=len(os.listdir(test_path))) as pbar:       
    #test all test images
    for img in os.listdir(test_path):
        label = ensemble_prediction(img, 'image')
        new_np_array = np.array([[img, label]])
        submission_data = np.concatenate((submission_data, new_np_array), axis=0)
        pbar.update(1)

#remove dummy
submission_data = np.delete(submission_data, 0, 0)

#save final submission
result_df = pd.DataFrame(submission_data, columns=['image','AdoptionSpeed_Pred'])
result_df.to_csv("/content/drive/My Drive/submission_.csv", index=False)

print('Done')

100%|██████████| 3644/3644 [1:55:37<00:00,  1.90s/it]
